In [1]:
!pip install Flask pyngrok line-bot-sdk requests openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5000
print(line_channel_access_token)
print("====")
print(line_channel_secret)

zKPBoejRjkFsB4q3iXWFQwMCybXeUorzi4Vhu5fa3ag+DwGtyCEK4bNplKRajqtED3u6iztG8boNYNX5ZbGyghZvkTp1ntIW72aPG9GWgaLuvFuxn6APOy7lEYTGo/gShXR8ZEzax961ZizVchxtcgdB04t89/1O/w1cDnyilFU=
====
01d18cc5b05062dd6fd8e3fecafb315d


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://397e-34-23-85-150.ngrok-free.app


In [5]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)
def get_completion(prompt):
    messages = [{"role": "system", "content": "以下用繁體中文台灣用語回答"},
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256

    )
    return response.choices[0].message.content

In [6]:
result = get_completion("簡介東吳大學")
print(result)

東吳大學是一所位於台灣台北市的知名私立大學，創立於1928年，是台灣歷史悠久的學府之一。東吳大學以人文學科為主，並擁有豐富的師資和研究資源。校園環境優美，教學設備完善，是許多學生夢寐以求的就讀學校之一。該校提供多元的學士、碩士和博士學位課程，並致力於培養學生的專業知識和實務能力。東吳大學也積極推動國際交流與合作，為學生提供豐富的國際


In [7]:
messages = []

In [8]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            messages.append(prompt)
            prompts = " ".join(messages)
            reply_text = get_completion(prompts)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            messages.append(reply_text)
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )


@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/May/2024 03:53:11] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510089614233960791","quoteToken":"8Y2jKbc5NbQAGIiP-Uy8hNbENdbsd7fjKBhDLbctXPknECH12voEntx9BmtpTCULxiNOyUdgKcPNxJfMO2kzjbvj8p2YiSAk3sG8z_6ZspEzLH9oM_Ogb8HrSn2-imuVhhUkbg3a5nKgYpla2RoPoQ","text":"AI 簡介東吳大學"},"webhookEventId":"01HYYN9QY0T48E3G12T5M8MJ7R","deliveryContext":{"isRedelivery":false},"timestamp":1716868406773,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"6d62d0240f9f46749b538e64c6d9429a","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 03:53:32] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510089637185192515","quoteToken":"scoZ8X306sc6OgJS0d965DM6sugPjIZiil1cFg4EI-HAAJ4CqwjzRj9m-WomBGZkkcmNXFh1RIaSJKR4_CBFJO4UinTiw_IlxxVcmV5FLi5q1WPL8_19IWyqYGpKTP4VXjOH5Kz1u-12GU3W4sJqxA","text":"AI 那城區部地址呢？"},"webhookEventId":"01HYYNA4V2FSC6RZB0J9VN5P6M","deliveryContext":{"isRedelivery":false},"timestamp":1716868420343,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"6acff7a4c24c484d85eb9aa1b932f618","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 03:53:42] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510089704142799045","quoteToken":"WYSshO2vLH-Qg3jS_YOOZxXPP1czsI7bs4wfOkuEUM4G45a56snyfpmE9GHT5SLQARqEy5pvSnhBFOjp253rd_fxvnkBYOaOQyEXe4rDgOrM5ghzuacZcn35I7u3-oBNAPe1fQR0r7JBaj-k3-8kxA","text":"AI 有什麼傑出校友？"},"webhookEventId":"01HYYNBCB0WP90FF69ERSFYTAG","deliveryContext":{"isRedelivery":false},"timestamp":1716868460388,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"ee125c5280d04f01819f1fef3bea72ec","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 03:54:26] "POST / HTTP/1.1" 200 -
